In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import AlgoLoop
import StatisticFunc

plt.style.use('Solarize_Light2')                     # 绘图风格
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 字体雅黑
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False    # 处理负号

# 预处理（不考虑WTI原油期货）

## 提取并处理数据

### 权益类资产换手率

In [2]:
Turnovers = pd.read_excel(io="Raw/资产换手率.xlsx")                      # 读取数据
Turnovers = Turnovers.drop([0], axis=0)                                   # 删除多余行
Turnovers["日期"] = pd.DatetimeIndex(Turnovers["日期"])                    # 类型转换
Turnovers = Turnovers.set_index("日期")                                   # 设置索引

FileNotFoundError: [Errno 2] No such file or directory: 'Raw/国内资产换手率.xlsx'

In [ ]:
Turnovers = Turnovers.drop(Turnovers.index[0], axis=0)                   # 日期对齐

### GC007利率

In [ ]:
Rates = pd.read_excel(io="Raw/GC007利率.xlsx")           #  读取数据
Rates = Rates.drop([0, 1, 2], axis=0)                   # 删除多余行
Rates.columns = ['日期', 'GC007']                        # 修改列名
Rates["日期"] = pd.DatetimeIndex(Rates["日期"])           # 类型转换
Rates = Rates.set_index("日期")                          # 设置索引

In [3]:
Rates = Rates.drop(Rates.index[0], axis=0)                     # 日期对齐

NameError: name 'Rates' is not defined

### 十年国债收益率

In [ ]:
BondRates = pd.read_excel(io="Raw/中债国债到期收益率(中债)(日).xls")        # 读取数据
BondRates.columns = ['日期', '十年国债收益率']                             # 修改列名
BondRates = BondRates.drop([0, 3870, 3871], axis=0)                     # 删除多余行
BondRates["日期"] = pd.DatetimeIndex(BondRates["日期"])                   # 类型转换
BondRates = BondRates.set_index("日期")                                  # 设置索引

### 各资产收盘价

In [ ]:
Assets = pd.read_excel(io="Raw/国内资产收盘价.xlsx")
Assets = Assets.drop([0], axis=0)                                   # 删除多余行
Assets["日期"] = pd.DatetimeIndex(Assets["日期"])                    # 类型转换
Assets = Assets.set_index("日期")                                   # 设置索引

In [ ]:
Assets.head()

### 十年国债价格指数

In [ ]:
Data = Assets.merge(BondRates, how='left', left_index=True, right_index=True)

In [ ]:
Data = Data.merge(Rates, how='left', left_index=True, right_index=True)

In [ ]:
Data['持有时长'] = (Data.index - Data.index[0]).days
Data['10年国债价格指数'] = 100 - (Data['十年国债收益率'] - 3.0) * 8.2 + 3.0 * Data['持有时长'] / 365.0
Data['10年国债价格指数'] = Data['10年国债价格指数'].fillna(method='ffill')

## 计算各资产收益率

- 去除WTI原油期货

### 日内损益

In [ ]:
# 利用自制10年国债价格指数，替换上证10年国债
Assets['上证10年国债'] = Data['10年国债价格指数'].values

In [ ]:
Returns = Assets.pct_change(axis=0)
Returns = Returns.dropna(axis=0, how='all')           # 删除无数据日

In [ ]:
Returns = Returns.drop('中信证券WTI原油期货', axis=1)
Returns.head()

### 累计损益

In [ ]:
cumReturns = (1.0 + Returns)
cumReturns = cumReturns.fillna(1.0)                    # 填充空值
cumReturns = cumReturns.cumprod()                      # 计算各资产累计收益率

In [ ]:
cumReturns.plot(figsize=(16, 8))

### 杠杆调整后的债券净值

In [ ]:
lever = 2.0                                 # 杠杆
leverReturns = Returns.copy()               # 创建副本 
leverCumReturns = cumReturns.copy()   

# 更新杠杆调整后的债券净值
leverReturns.loc[:, ['上证10年国债', '信用债3-5AAA']] = Returns.loc[:, ['上证10年国债', '信用债3-5AAA']] * (1.0 + lever) - Rates.values * lever / (365.0 * 100)

# 更新累计净值
leverCumReturns = (1.0 + leverReturns)
leverCumReturns = leverCumReturns.fillna(1.0)                    # 填充空值
leverCumReturns = leverCumReturns.cumprod()                      # 计算各资产累计收益率

# 风险平价 + 杠杆策略（指数权重EMA）

## 程序运行

In [ ]:
tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, mode='ema')

In [ ]:
StatisticFunc.WeightPlot(tradeDF, weightDF, 'D')

## 回测表现汇总

In [ ]:
# 表现汇总
years = np.unique(tradeDF.reset_index()['日期'].apply(lambda x:str(x)[:4])) 

SummaryDF = pd.DataFrame(index=years)

SummaryDF["年收益率"] = StatisticFunc.AnnualReturns( pd.DataFrame(tradeDF['投资组合净值']) )
SummaryDF["年波动率"] = StatisticFunc.AnnualVolatility(pd.DataFrame(tradeDF['投资组合净值'])).values
SummaryDF["无基准夏普比率"] = SummaryDF['年收益率'] / SummaryDF['年波动率']
SummaryDF["最大回撤"] = StatisticFunc.AnnualMaxDrawdown(pd.DataFrame(tradeDF['最大回撤'])).values

SummaryDF

## 收益率贡献度

In [ ]:
annualContrb = StatisticFunc.AnnualContribution(tradeDF=tradeDF, weightDF=weightDF, assetDF=leverCumReturns)

In [ ]:
StatisticFunc.BarPlot(annualContrb, 'D')

## 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, label='每120个交易日调仓')
    
plt.plot(cumReturns.index, cumReturns['中证500'], label='中证500')
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 3.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均）')
plt.savefig('净值曲线（杠杆，指数平均）D.png')

# 基准组合（横截面动量+时序动量+换手率+铜金+铜油）

## 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':True, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':True, 
            'copperGold':True, 'copperGas':True}

tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, mode='ema', 
                                       dt=120, up=0.50, thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF, weightDF, '杠杆+横截面动量+时序动量+换手率+铜金+铜油D')

## 回测表现汇总

In [ ]:
# 表现汇总
years = np.unique(tradeDF.reset_index()['日期'].apply(lambda x:str(x)[:4])) 

SummaryDF = pd.DataFrame(index=years)

SummaryDF["年收益率"] = StatisticFunc.AnnualReturns( pd.DataFrame(tradeDF['投资组合净值']) )
SummaryDF["年波动率"] = StatisticFunc.AnnualVolatility(pd.DataFrame(tradeDF['投资组合净值'])).values
SummaryDF["无基准夏普比率"] = SummaryDF['年收益率'] / SummaryDF['年波动率']
SummaryDF["最大回撤"] = StatisticFunc.AnnualMaxDrawdown(pd.DataFrame(tradeDF['最大回撤'])).values

SummaryDF

## 收益率贡献度

In [ ]:
annualContrb = StatisticFunc.AnnualContribution(tradeDF=tradeDF, weightDF=weightDF, assetDF=leverCumReturns)

In [ ]:
StatisticFunc.BarPlot(annualContrb, '横截面动量+时序动量+换手率+铜金+铜油D')

## 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, label='横截面动量+时序动量+换手率+铜金+铜油')
    
plt.plot(cumReturns.index, cumReturns['中证500'], label='中证500')
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均，横截面动量+时序动量+换手率+铜金+铜油）')
plt.savefig('净值曲线（杠杆，指数平均，横截面动量+时序动量+换手率+铜金+铜油）D.png')

# 风险平价 + 杠杆 + 多因子策略（指数平均）

## 时序动量+换手率+铜金+铜油

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':True, 
            'reverseX':False, 'reverseT':False,
            'turnover':True, 
            'copperGold':True, 'copperGas':True}

tradeDF_1, weightDF_1 = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, mode='ema', 
                                       dt=120, up=0.50, thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_1, weightDF_1, '杠杆+时序动量+换手率+铜金+铜油D')

### 回测表现汇总

In [ ]:
# 表现汇总
years = np.unique(tradeDF_1.reset_index()['日期'].apply(lambda x:str(x)[:4])) 

SummaryDF_1 = pd.DataFrame(index=years)

SummaryDF_1["年收益率"] = StatisticFunc.AnnualReturns( pd.DataFrame(tradeDF_1['投资组合净值']) )
SummaryDF_1["年波动率"] = StatisticFunc.AnnualVolatility(pd.DataFrame(tradeDF_1['投资组合净值'])).values
SummaryDF_1["无基准夏普比率"] = SummaryDF_1['年收益率'] / SummaryDF_1['年波动率']
SummaryDF_1["最大回撤"] = StatisticFunc.AnnualMaxDrawdown(pd.DataFrame(tradeDF_1['最大回撤'])).values

SummaryDF_1

### 收益率贡献度

In [ ]:
annualContrb_1 = StatisticFunc.AnnualContribution(tradeDF=tradeDF_1, weightDF=weightDF_1, assetDF=leverCumReturns)

In [ ]:
StatisticFunc.BarPlot(annualContrb_1, '时序动量+换手率+铜金+铜油D')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_1.index, tradeDF_1['投资组合净值']/10000, label='时序动量+换手率+铜金+铜油')

# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='五因子基准组合')    
plt.plot(cumReturns.index, cumReturns['中证500'], label='中证500')
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均，时序动量+换手率+铜金+铜油）')
plt.savefig('净值曲线（杠杆，指数平均，时序动量+换手率+铜金+铜油）D.png')

## 横截面动量+换手率+铜金+铜油

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':True, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':True, 
            'copperGold':True, 'copperGas':True}

tradeDF_2, weightDF_2 = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, mode='ema', 
                                       dt=120, up=0.50, thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_2, weightDF_2, '杠杆+横截面动量+换手率+铜金+铜油D')

### 回测表现汇总

In [ ]:
# 表现汇总
years = np.unique(tradeDF_2.reset_index()['日期'].apply(lambda x:str(x)[:4])) 

SummaryDF_2 = pd.DataFrame(index=years)

SummaryDF_2["年收益率"] = StatisticFunc.AnnualReturns( pd.DataFrame(tradeDF_2['投资组合净值']) )
SummaryDF_2["年波动率"] = StatisticFunc.AnnualVolatility(pd.DataFrame(tradeDF_2['投资组合净值'])).values
SummaryDF_2["无基准夏普比率"] = SummaryDF_2['年收益率'] / SummaryDF_2['年波动率']
SummaryDF_2["最大回撤"] = StatisticFunc.AnnualMaxDrawdown(pd.DataFrame(tradeDF_2['最大回撤'])).values

SummaryDF_2

### 收益率贡献度

In [ ]:
annualContrb_2 = StatisticFunc.AnnualContribution(tradeDF=tradeDF_2, weightDF=weightDF_2, assetDF=leverCumReturns)

In [ ]:
StatisticFunc.BarPlot(annualContrb_2, '横截面动量+换手率+铜金+铜油D')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_2.index, tradeDF_2['投资组合净值']/10000, label='横截面动量+换手率+铜金+铜油')
        
# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='五因子基准组合')  
plt.plot(cumReturns.index, cumReturns['中证500'], label='中证500')
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均，横截面动量+换手率+铜金+铜油）')
plt.savefig('净值曲线（杠杆，指数平均，横截面动量+换手率+铜金+铜油）D.png')

## 横截面动量+时序动量+铜金+铜油

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':True, 'momentumT':True, 
            'reverseX':False, 'reverseT':False,
            'turnover':False, 
            'copperGold':True, 'copperGas':True}

tradeDF_3, weightDF_3 = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, mode='ema', 
                                       dt=120, up=0.50, thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_3, weightDF_3, '杠杆+横截面动量+时序动量+铜金+铜油D')

### 回测表现汇总

In [ ]:
# 表现汇总
years = np.unique(tradeDF_3.reset_index()['日期'].apply(lambda x:str(x)[:4])) 

SummaryDF_3 = pd.DataFrame(index=years)

SummaryDF_3["年收益率"] = StatisticFunc.AnnualReturns( pd.DataFrame(tradeDF_3['投资组合净值']) )
SummaryDF_3["年波动率"] = StatisticFunc.AnnualVolatility(pd.DataFrame(tradeDF_3['投资组合净值'])).values
SummaryDF_3["无基准夏普比率"] = SummaryDF_3['年收益率'] / SummaryDF_3['年波动率']
SummaryDF_3["最大回撤"] = StatisticFunc.AnnualMaxDrawdown(pd.DataFrame(tradeDF_3['最大回撤'])).values

SummaryDF_3

### 收益率贡献度

In [ ]:
annualContrb_3 = StatisticFunc.AnnualContribution(tradeDF=tradeDF_3, weightDF=weightDF_3, assetDF=leverCumReturns)

In [ ]:
StatisticFunc.BarPlot(annualContrb_3, '横截面动量+时序动量+铜金+铜油D')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_3.index, tradeDF_3['投资组合净值']/10000, label='横截面动量+时序动量+铜金+铜油')
    
# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='五因子基准组合')  
plt.plot(cumReturns.index, cumReturns['中证500'], label='中证500')
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均，横截面动量+时序动量+铜金+铜油）')
plt.savefig('净值曲线（杠杆，指数平均，横截面动量+时序动量+铜金+铜油）D.png')

## 横截面动量+时序动量+换手率+铜油

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':True, 'momentumT':True, 
            'reverseX':False, 'reverseT':False,
            'turnover':True, 
            'copperGold':False, 'copperGas':True}

tradeDF_4, weightDF_4 = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, mode='ema', 
                                       dt=120, up=0.50, thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_4, weightDF_4, '杠杆+横截面动量+时序动量+换手率+铜油D')

### 回测表现汇总

In [ ]:
# 表现汇总
years = np.unique(tradeDF_4.reset_index()['日期'].apply(lambda x:str(x)[:4])) 

SummaryDF_4 = pd.DataFrame(index=years)

SummaryDF_4["年收益率"] = StatisticFunc.AnnualReturns( pd.DataFrame(tradeDF_4['投资组合净值']) )
SummaryDF_4["年波动率"] = StatisticFunc.AnnualVolatility(pd.DataFrame(tradeDF_4['投资组合净值'])).values
SummaryDF_4["无基准夏普比率"] = SummaryDF_4['年收益率'] / SummaryDF_4['年波动率']
SummaryDF_4["最大回撤"] = StatisticFunc.AnnualMaxDrawdown(pd.DataFrame(tradeDF_4['最大回撤'])).values

SummaryDF_4

### 收益率贡献度

In [ ]:
annualContrb_4 = StatisticFunc.AnnualContribution(tradeDF=tradeDF_4, weightDF=weightDF_4, assetDF=leverCumReturns)

In [ ]:
StatisticFunc.BarPlot(annualContrb_4, '横截面动量+时序动量+换手率+铜油D')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_4.index, tradeDF_4['投资组合净值']/10000, label='横截面动量+时序动量+换手率+铜油')
    
    
# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='五因子基准组合')  
plt.plot(cumReturns.index, cumReturns['中证500'], label='中证500')
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均，横截面动量+时序动量+换手率+铜油）')
plt.savefig('净值曲线（杠杆，指数平均，横截面动量+时序动量+换手率+铜油）D.png')

## 横截面动量+时序动量+换手率+铜金

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':True, 'momentumT':True, 
            'reverseX':False, 'reverseT':False,
            'turnover':True, 
            'copperGold':True, 'copperGas':False}

tradeDF_5, weightDF_5 = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, Turnovers, mode='ema', 
                                       dt=120, up=0.50, thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_5, weightDF_5, '杠杆+横截面动量+时序动量+换手率+铜金D')

### 回测表现汇总

In [ ]:
# 表现汇总
years = np.unique(tradeDF_5.reset_index()['日期'].apply(lambda x:str(x)[:4])) 

SummaryDF_5 = pd.DataFrame(index=years)

SummaryDF_5["年收益率"] = StatisticFunc.AnnualReturns( pd.DataFrame(tradeDF_5['投资组合净值']) )
SummaryDF_5["年波动率"] = StatisticFunc.AnnualVolatility(pd.DataFrame(tradeDF_5['投资组合净值'])).values
SummaryDF_5["无基准夏普比率"] = SummaryDF_5['年收益率'] / SummaryDF_5['年波动率']
SummaryDF_5["最大回撤"] = StatisticFunc.AnnualMaxDrawdown(pd.DataFrame(tradeDF_5['最大回撤'])).values

SummaryDF_5

### 收益率贡献度

In [ ]:
annualContrb_5 = StatisticFunc.AnnualContribution(tradeDF=tradeDF_5, weightDF=weightDF_5, assetDF=leverCumReturns)

In [ ]:
StatisticFunc.BarPlot(annualContrb_5, '横截面动量+时序动量+换手率+铜金D')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_5.index, tradeDF_5['投资组合净值']/10000, label='横截面动量+时序动量+换手率+铜金')

# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='五因子基准组合')  
plt.plot(cumReturns.index, cumReturns['中证500'], label='中证500')
plt.plot(cumReturns.index, cumReturns['信用债3-5AAA'], label='信用债3-5AAA')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均，横截面动量+时序动量+换手率+铜金）')
plt.savefig('净值曲线（杠杆，指数平均，横截面动量+时序动量+换手率+铜金)D.png')

# 汇总比较

In [ ]:
fig = plt.figure(figsize=(16, 8))



plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='五因子基准组合')

plt.plot(tradeDF_1.index, tradeDF_1['投资组合净值']/10000, label='时序动量+换手率+铜金+铜油')
plt.plot(tradeDF_2.index, tradeDF_2['投资组合净值']/10000, label='横截面动量+换手率+铜金+铜油')
plt.plot(tradeDF_3.index, tradeDF_3['投资组合净值']/10000, label='横截面动量+时序动量+铜金+铜油')
plt.plot(tradeDF_5.index, tradeDF_4['投资组合净值']/10000, label='横截面动量+时序动量+换手率+铜油')
plt.plot(tradeDF_5.index, tradeDF_5['投资组合净值']/10000, label='横截面动量+时序动量+换手率+铜金')


plt.xlabel('时间')
plt.ylabel('净值')
plt.legend(loc='upper left')
plt.title('各多因子模型净值曲线')
plt.savefig('各多因子模型净值曲线D.png')